# ETL Global temperature data & Coral reef data set

In this notebook you will enconter the initial process for the project. We will be (E)xtracting (T)ransforming and (L)oading the data to be able to tell a story with data. 

Developers: Jorge Daniel Atuesta, John Cass & John Strode



In [1]:
#Importing dependencies
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
#Reading in the CSV
file_path = "data/GlobalTemperatures.csv"
gt_df = pd.read_csv(file_path)
gt_df.tail()

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
3187,2015-08-01,14.755,0.072,20.699,0.110,9.005,0.170,17.589,0.057
3188,2015-09-01,12.999,0.079,18.845,0.088,7.199,0.229,17.049,0.058
3189,2015-10-01,10.801,0.102,16.450,0.059,5.232,0.115,16.290,0.062
3190,2015-11-01,7.433,0.119,12.892,0.093,2.157,0.106,15.252,0.063
3191,2015-12-01,5.518,0.100,10.725,0.154,0.287,0.099,14.774,0.062


# Understading the data set

This data set has a monthly recolation of Average Global temperature dating all the way back to 1750. The temperature is mesuared both for global land temperature and ocean average temperature. 

We will proceed to Extract, Transform and Load the data.

Extract: We extracted the dataa from the followign link: https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data

Transform: We will clean the data to only show the following:

    1. land and ocean average temperature 
    2. Record said temperatures in decades from 1980 to 2015
        2.1 1975
        2.2 1985
        2.3 1995
        2.4 2005
        2.5 2015
    3. Since the data is set in monthly records we will average the years temp
    4. Drop all the duplicates adn NaN values inside the df
    5. Merge the two data sets. We will do a left join on the data column for both data sets. 
    
    
Load: We will load the final dataframe to postgres for furthure action. 
        


In [3]:
#Getting the values in the data set using .count()
gt_df.count()
# As shown below the data set has inconsisten data that might be because of NaN values or duplicates. 

dt                                           3192
LandAverageTemperature                       3180
LandAverageTemperatureUncertainty            3180
LandMaxTemperature                           1992
LandMaxTemperatureUncertainty                1992
LandMinTemperature                           1992
LandMinTemperatureUncertainty                1992
LandAndOceanAverageTemperature               1992
LandAndOceanAverageTemperatureUncertainty    1992
dtype: int64

In [4]:
#Adjusting the columns in the df to match what we need
#We will start by only haveing two columns : Date and land and Ocean Temp.
updated_gt_df = gt_df[["dt", "LandAndOceanAverageTemperature"]].rename(columns={"dt": "date", "AverageTemperature": "avg_global_temp"})
updated_gt_df.head()

,date,LandAndOceanAverageTemperature
0,1750-01-01,NaN
1,1750-02-01,NaN
2,1750-03-01,NaN
3,1750-04-01,NaN
4,1750-05-01,NaN


In [5]:
updated_gt_df.count()

date                              3192
LandAndOceanAverageTemperature    1992
dtype: int64

In [6]:
#Check the total amount of NaN values in the data set
#Link for checking null values in df using pandas and numpy: https://chartio.com/resources/tutorials/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe/
updated_gt_df.isnull().sum()
#Because there are NAN values (1200) in the landandoceanavgtemp we need to remove them from the data set. We can see that the dates column has no NaN values

date                                 0
LandAndOceanAverageTemperature    1200
dtype: int64

In [7]:
#We now are going to drop any NaN values from the data set.
#Here is a link to the documentation for dropping NaN values: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
no_nan_gt_df = updated_gt_df.dropna()
no_nan_gt_df.head()


,date,LandAndOceanAverageTemperature
1200,1850-01-01,12.833
1201,1850-02-01,13.588
1202,1850-03-01,14.043
1203,1850-04-01,14.667
1204,1850-05-01,15.507


***Its important to note that now the df dosent show the any years previouse to 1850 this is because they had no data points or records on file and where dropped. This dosent affect our process as we will be looking at the following years: 1975-1985-1995-2005-2015***

In [8]:
#Check if therea are duplicated valuesvalues on the date in the df
#link: https://thispointer.com/pandas-find-duplicate-rows-in-a-dataframe-based-on-all-or-selected-columns-using-dataframe-duplicated-in-python/

duplicateRowsDF = no_nan_gt_df[no_nan_gt_df.duplicated(['date'])]

print("Duplicate Rows based on a single column are:", duplicateRowsDF, sep='\n')

#We can see in the out put no duplciates where found in the data set. 

Duplicate Rows based on a single column are:
Empty DataFrame
Columns: [date, LandAndOceanAverageTemperature]
Index: []


In [9]:
#Reset the index to start from 0
ri_gt_df = no_nan_gt_df.reset_index()
almost_gt_df = ri_gt_df.drop(columns=['index'])
almost_gt_df.head()


,date,LandAndOceanAverageTemperature
0,1850-01-01,12.833
1,1850-02-01,13.588
2,1850-03-01,14.043
3,1850-04-01,14.667
4,1850-05-01,15.507


In [10]:
#We need to add a column with date time format and delete the preiviouse date column 
almost_gt_df['date'] = pd.to_datetime(almost_gt_df['date'])


In [11]:
#Adding the columns to the df
almost_gt_df["year"] = almost_gt_df['date'].dt.year
almost_gt_df["month"] =almost_gt_df['date'].dt.month
almost_gt_df["day"] = almost_gt_df['date'].dt.day
a_df = almost_gt_df
a_df

,date,LandAndOceanAverageTemperature,year,month,day
0,1850-01-01,12.833,1850,1,1
1,1850-02-01,13.588,1850,2,1
2,1850-03-01,14.043,1850,3,1
3,1850-04-01,14.667,1850,4,1
4,1850-05-01,15.507,1850,5,1
...,...,...,...,...,...
1987,2015-08-01,17.589,2015,8,1
1988,2015-09-01,17.049,2015,9,1
1989,2015-10-01,16.290,2015,10,1
1990,2015-11-01,15.252,2015,11,1


In [12]:
#Removing unwanted columns
df1 = a_df.drop(columns=['date','month','day'])
df1

,LandAndOceanAverageTemperature,year
0,12.833,1850
1,13.588,1850
2,14.043,1850
3,14.667,1850
4,15.507,1850
...,...,...
1987,17.589,2015
1988,17.049,2015
1989,16.290,2015
1990,15.252,2015


In [13]:
#using group by function 
df2 = df1.groupby(['year'])
df2

In [14]:
#Getting average temp per year of all the data set. 
agg_df = df2.agg({
    'LandAndOceanAverageTemperature': 'mean'
})
agg_df.head()

,LandAndOceanAverageTemperature
year,
1850,14.867167
1851,14.991833
1852,15.006500
1853,14.955167
1854,14.991000


In [56]:
#We need to filter to only gather the data for the years we need.
# Years we need = 1975-1985-1995-2005-2015

# year_1975_df = almost_gt_df.loc[almost_gt_df['date'] == "1975-01-01"]
# year_1975_df.head()

# Loading the data into postgres

In [ ]:
#STEP 7 
#Creating connection string and engine
#Create DB in postgres name it coral_reef_db
#Create the tables on postgres
connection_string = "postgres:postgres@localhost:5432/coral_reef_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
#STEP 8 
# Confirm tables
engine.table_names()

In [ ]:
#STEP 9
#Loading data into postgres before the . put the df name and inside the () in name put the name of the table from the data base where the data will be placed.
.to_sql(name='', con=engine, if_exists='append', index=True)

In [ ]:
#STEP 9
#Loading data into postgres
.to_sql(name='', con=engine, if_exists='append', index=True)